In [1]:
import requests

In [2]:
import geopandas as gpd
import pandas as pd

## Open County data

In [3]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_gdf = gpd.read_file(file_path)

## Get Language Data

In [4]:
r = requests.get("https://api.census.gov/data/2023/acs/acs5/groups/B16001.json")
columns_obj = r.json()

### Get columns to query and rename for later

In [5]:
columns = []
rename_vars = {}
variables = columns_obj["variables"]
for name, variable in list(variables.items()):
    v_split = variable["label"].split("!!")
    if len(v_split) != 3:
        continue

    if v_split[0] == "Estimate":
        label = v_split[-1]
        rename_vars[name] = label

    if name.endswith("E"):
        columns.append(name)

In [6]:
columns.remove("B16001_002E")  # Remove English
columns.remove("B16001_003E")  # Remove spanish

In [7]:
columns.append("GEO_ID")
columns_formatted = ",".join(columns)

In [8]:
response = requests.get(
    f"https://api.census.gov/data/2023/acs/acs5?get={columns_formatted}&for=state:*"
)

In [9]:
data = response.json()
columns = data[0]
rows = data[1:]
language_df = pd.DataFrame(rows, columns=columns)

In [10]:
estimate_cols = [col for col in language_df.columns if col.endswith("E")]

In [11]:
language_formtted_df = language_df[["GEO_ID", *estimate_cols]]
language_formtted_df[estimate_cols] = language_formtted_df[estimate_cols].astype(int)

/tmp/ipykernel_13827/280287716.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  language_formtted_df[estimate_cols] = language_formtted_df[estimate_cols].astype(int)


In [12]:
language_formtted_df["most_common_language_raw"] = language_formtted_df[
    estimate_cols
].idxmax(axis=1)

In [13]:
def check_language(row) -> str:
    ethnicity_col = row["most_common_language_raw"]
    val = row[ethnicity_col]
    return variables[ethnicity_col]["label"].split("!!")[-1][:-1]

In [14]:
language_formtted_df["most_common_language"] = language_formtted_df.apply(
    lambda row: check_language(row), axis=1
)

In [15]:
rename_vars["GEO_ID"] = "AFFGEOID"
language_formtted_df = language_formtted_df.rename(columns=rename_vars)

## Merge Data

In [16]:
language_gdf = states_gdf.merge(language_formtted_df, on="AFFGEOID", how="inner")

In [17]:
language_gdf = language_gdf.to_crs(9311)
language_gdf.to_file("data/Language_Spoken_At_Home_Not_English_Spanish.gpkg")

In [18]:
language_gdf.groupby("most_common_language").size().reset_index(
    name="COUNT"
).sort_values("COUNT", ascending=False)

,most_common_language,COUNT
2,"Chinese (incl. Mandarin, Cantonese)",13
4,German,7
3,French (incl. Cajun),6
13,Vietnamese,6
1,Arabic,4
11,Portuguese,3
7,"Ilocano, Samoan, Hawaiian, or other Austronesi...",2
14,"Yiddish, Pennsylvania Dutch or other West Germ...",2
9,Other Native languages of North America,2
8,Navajo,2
